In [ ]:
# Can we do the same with Transforms?? is it a better idea?
# Does it suffer the same type issues?

In [ ]:
from fastai2.basics import *

In [ ]:
def _maintain_labels(old, new):
    if hasattr(old, 'labels'): new.labels = old.labels

In [ ]:
_old_call = Transform._call
@patch
def _call(self:Transform, fn, x, **kwargs):
    res = _old_call(self, fn, x, **kwargs)
    _maintain_labels(x, res)
    return res

In [ ]:
def _add_labels(f):
    def _inner(x):
        x.labels = getattr(x,'labels',[])
        x.labels.append(f(x))
    return _inner

In [ ]:
def subscribe(tfm):
    old_call = tfm._call
    def _inner(f):
        def _call(fn, x, **kwargs):
            res = old_call(fn, x, **kwargs)
            if res is not x: f(res)
            return res
        tfm._call = _call
        return f
    return _inner

In [ ]:
def labeller(tfm):
    def _inner(f):
        return subscribe(tfm)(_add_labels(f))
    return _inner

In [ ]:
@Transform
def neg(x:Tensor): return -x
@Transform
def add_greeting(x:str): return 'hello ' + x

In [ ]:
CAT1,CAT2 = 'cat1','cat2'

In [ ]:
@labeller(neg)
def labeller_cat1(x): return CAT1
@labeller(neg)
def labeller_cat2(x): return CAT2

In [ ]:
@labeller(add_greeting)
def labeller_greeting(x): return CAT1

In [ ]:
x = tensor(2)
tfms = Pipeline([neg, add_greeting])
res = tfms(x)

In [ ]:
res.labels

['cat1', 'cat2']

In [ ]:
class MyStr(str): pass
class MyInt(int): pass

In [ ]:
@Transform
def str2int(x:MyStr): return MyInt(float(x))
@Transform
def int2tensor(x:MyInt): return tensor(x)

In [ ]:
@labeller(str2int)
def func1(x): return 'str2int'
@labeller(int2tensor)
def func2(x):
    if x>2: return 'int2tensor'

In [ ]:
pipe = Pipeline([str2int, int2tensor])

In [ ]:
res = pipe(MyStr('1.2'))
res.labels

['str2int', None]

## IMDB tests

In [ ]:
from fastai2.text.all import *

In [ ]:
source = untar_data(URLs.IMDB_SAMPLE)

In [ ]:
df = pd.read_csv(source/'texts.csv')

In [ ]:
splits = ColSplitter('is_valid')(df)

In [ ]:
col_reader = mk_transform(ColReader('text'))
my_str = mk_transform(lambda o: MyStr(o))
tkzer = Tokenizer.from_df('text')
nmzer = Numericalize()

In [ ]:
pipe = Pipeline([col_reader,my_str,tkzer])
pipe.setup(df)

In [ ]:
POS,NEG = 'positive','negative'

In [ ]:
@labeller(my_str)
def liked(x):
    return POS
#     if 'liked' in x: return POS

In [ ]:
dset = Datasets(df, tfms=[[col_reader, my_str, tkzer, nmzer]])

In [ ]:
res = dset[0][0]

In [ ]:
res

TensorText([   2,   36, 2119,    0,   34, 3421,   65,    0,   65,   87, 3422])

In [ ]:
res.labels

['positive']